# 202202_InteractiveRun

## Setup

In [1]:
### Setup Env for Local
import os; from importlib import reload;
%cd {os.environ['PROJECT_PATH']}
from utils.constants import *

/app


In [2]:
### Python Imports
import json, sys
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import jupyter_dash

from dash_app import cache
from dash_app.cache import redis_store
from utils import constants
from data import load_data, preprocess

# def log(*msg): print(*msg, file=sys.stderr)
# def log(*msg): print("nipams-log",*msg)
# LOGS = []
# def log(*msg): 
#     LOGS.extend(msg)


/app/src/dash_app/cache.py:49: UserWarning: Using FakeRedis - Not suitable for Production Use.
  warnings.warn('Using FakeRedis - Not suitable for Production Use.')


In [6]:
dcc.Store(id='asdf',storage_type='session')

TypeError: The `dcc.Store` component (version 2.2.0) with the ID "asdf" received an unexpected keyword argument: `persistence`
Allowed arguments: clear_data, data, id, modified_timestamp, storage_type

## Declare JupyterApp

In [ ]:
### App Setup
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

LAYOUT = [
    html.H1('Data Loading', style={"textAlign": "center"})
]

app.layout = dbc.Col(LAYOUT);
# app.run_server(mode='inline',port=8501, host="127.0.0.1", debug=True, **{'width': '200px', 'height': '100px'})
app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '200px', 'height': '100px'})

# Read Data

In [ ]:
### Load Data
from data import load_data, preprocess; reload(load_data)
# dfAll = load_data.load_dataframe_from_pickle('data/interim')

app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

TEST_TYPES = ['HLV','LLV','rec','res','bre']
LOAD_LAYOUT = [dbc.Card(
    dbc.CardBody([
        html.H4("Load VCG Data", className="card-title"),
        html.P(
            "Select which types of VCG readings to work with. (Read preprocessed VCG data from local file system.)",
            className="card-text",
        ),
        dbc.InputGroup([
            dcc.Dropdown(id='input_test_types', multi=True,
                         options=[{'value':i,'label':i} for i in TEST_TYPES], value=TEST_TYPES[:2],
                        style={'minWidth':'400px'}),
            dbc.Button(id='button-load', children=['Load Data'], n_clicks=0, color="primary")
        ]),
        dbc.Alert("Success! Your data loaded Successfully",id="data-load-success",dismissable=True,fade=True,duration=2000,is_open=False,),
        # html.Pre('\n'.join(['asdf','asdf2','qwer']), style={'overflow-y':'scroll', 'max-height':'100px', 'background':'grey'}),
        dcc.Store('raw-data'),
    ])
)]

@app.callback(
    [Output('raw-data','data'), Output('data-load-success','is_open')],
    State('input_test_types','value'), Input('button-load','n_clicks') 
)
def button_clicked(types, n):
    print('input-file-pattern, button-load', types, n)
    dfAll = load_data.load_dataframe_from_pickle('data/interim', '|'.join(types or []))
    key = cache.set(dfAll)
    print('set cache', key)
    return key, True


app.layout = dbc.Col(LOAD_LAYOUT);
app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '600px', 'height': '400px'})

# Dataset Overview

In [ ]:
from dash_app.components import datatable_comp; reload(datatable_comp)
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Card([
        dbc.CardBody([
            html.H4("Load VCG Data", className="card-title"),
            html.P(
                "Select which types of VCG readings to work with. (Read preprocessed VCG data from local file system.)",
                className="card-text",)
        ]),
        html.Div(id='datatable-container'),
        html.Div(id='graphs')
])

app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '600px'})

In [ ]:
### App Setup
from dash_app.components import datatable_comp; reload(datatable_comp)
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dcc.Dropdown(
        ['iris', 'gapminder'],
        'iris',
        id='dataset-selector'
    ),
    html.Div(id='datatable-container'),
    html.Div(id='graphs')
])

# Display a DataTableIO based off of the dataset
@callback(
    Output('datatable-container', 'children'),
    Input('dataset-selector', 'value'))
def display_datatable(value):
    df = getattr(px.data, value)()
    return datatable_comp.DataTableAIO(df, aio_id='my-dataset')

# Graph the filtered data
@callback(
    Output('graphs', 'children'),
    Input(datatable_comp.DataTableAIO.ids.datatable('my-dataset'), 'filter_query'),
    State(datatable_comp.DataTableAIO.ids.store('my-dataset'), 'data')
)
def update_graph(filter_query, store):
    # Fetch the data that `DataTableAIO` stored in Redis
    df = redis_store.load(store['df'])
    # Filter the data using the method that `DataTableAIO` exposes
    # and uses internally
    df = datatable_comp.DataTableAIO.filter_df(df, filter_query)

    # Graph each column of the data
    divs = html.Div([
        html.Div([
            html.B(c),
            dcc.Graph(figure=px.histogram(x=df[c]))
        ]) for c in df.columns
    ])
    return divs


app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '600px'})

In [ ]:
## Dataset Overview

def summarize_data_bp(df):
    df = pd.concat([
        df.describe(),
        df.agg(['nunique'])
    ], axis=0)
    return df.round(3).reset_index()



# summarize_data_bp(dfAll)

In [ ]:
### Dataset Overview
# app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

tab1_content = dbc.CardBody([
        html.H6("Global Totals", className="card-title"),
        html.P("Overview of IMU and BP columns.",className="card-text"),
        html.Div(id='viz_overview',children=[dbc.Spinner(color="primary")])
    ])
tab2_content = dbc.CardBody([
        html.H6("Patient Totals", className="card-title"),
        html.P("Overview of IMU and BP columns.",className="card-text"),
        html.Div(id='viz_overview_patients',children=[dbc.Spinner(color="primary")])
    ])

OVERVIEW_LAYOUT = [dbc.Card(
    dbc.CardBody([
        html.H4("Overview VCG Data", className="card-title"),
        html.P("Overview of IMU and BP columns.",className="card-text",),
        dbc.Tabs([
                dbc.Tab(tab1_content, label="Overview"),
                dbc.Tab(tab2_content, label="Patient Totals"),
            ],id="tabs"),
        # dbc.Alert("Success! Your data loaded Successfully",id="overview-success",dismissable=True,fade=True,duration=2000,is_open=False,),
        # html.Pre('\n'.join(['asdf','asdf2','qwer']), style={'overflow-y':'scroll', 'max-height':'100px', 'background':'grey'}),
    ])),
]

@app.callback(Output('viz_overview','children'), Input('raw-data','data'))
def display_overview(df_key):
    print('display_overview', df_key)
    dfAll = cache.get(df_key)
    df = summarize_data_bp(dfAll)
    return dash_table.DataTable(
        df.to_dict('records'),
        [{"name": i, "id": i} for i in df.columns]
    )

@app.callback(Output('viz_overview_patients','children'), Input('raw-data','data'))
def viz_overview_patients(df_key):
    print('viz_overview_patients', df_key)
    dfAll = cache.get(df_key)
    df = dfAll.groupby('patient').agg(['count','nunique','min','max','mean']).round(2).reset_index()
    return dash_table.DataTable(
        df.to_dict('records'),
        [{"name": i, "id": i} for i in df.columns]
    )

# FORCE_CACHE_INPUT = [dcc.Store('raw-data'),dcc.Input(id='force-cache-input', value='', placeholder='Enter CacheKey..')]
# @app.callback(Output('raw-data','data'), Input('force-cache-input', 'value'))
# def update_cache(key):return cache.set(cache.get(key))

app.layout = dbc.Col(
    # FORCE_CACHE_INPUT + 
    LOAD_LAYOUT +
    OVERVIEW_LAYOUT);
app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, verbose=True, **{'width': '800px', 'height': '400px'})

##### 

In [ ]:
STOP_HERE

In [ ]:
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

FORCE_CACHE_INPUT = [dcc.Store('raw-data'),dcc.Input(id='force-cache-input', value='', placeholder='Enter CacheKey..')]
@app.callback(Output('raw-data','data'), Input('force-cache-input', 'value'))
def update_cache(key):return cache.set(cache.get(key))

app.layout = dbc.Col(FORCE_CACHE_INPUT);
app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '600px', 'height': '400px'})

In [ ]:
## Patient Overview
dfAll.groupby('patient').agg(['count','nunique','min','max','mean'])


In [ ]:
## Interactive - Load & Summarize



# Review Dataset

# Full Interactive App

In [ ]:
app.layout = html.Div(APP_LAYOUT)
app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '600px', 'height': '400px'})

In [ ]:
STOP_HERE

In [ ]:
### Imports
import json, sys
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State

from dash_app import cache

from utils import constants
from data import load_data, preprocess

# def log(*msg): print(*msg,file=sys.stderr)
def log(*msg): print("nipams-log",*msg)

### App Setup
app = jupyter_dash.JupyterDash(__name__, 
                               # assets_url_path=os.getcwd() + 'src/dash_app/assets/'
                               external_stylesheets=[dbc.themes.BOOTSTRAP]
                              )

app.layout = html.Div([
    html.H1('Dash & DataCaching', style={"textAlign": "center"}),
    html.H3('Load Data', style={"textAlign": "center"}),
    dcc.Input(id='input-file-pattern', value='LLV'),
    html.Button(id='button-load', children=['Load Data'], n_clicks=0),
    dcc.Store('raw-data'),
    
    html.H3('Visualize', style={"textAlign": "center"}),
    dcc.Dropdown(id='file_ids', multi=True,
                 options=[{'value':i,'label':i} for i in dfAll.file.unique()]),
    html.Div(id='viz'),
    dcc.Store('split-data'),
    
    html.H3('Modeling', style={"textAlign": "center"}),
    html.Button(id='button-train', children=['Train Model'], n_clicks=0),
    dcc.Store('model-binary'),
    
    html.Div(id='logs-div'),
])


### Load Data
@app.callback(
    Output('raw-data','data'),
    Input('input-file-pattern','value'), Input('button-load','n_clicks') 
)
def button_clicked(pattern, n):
    log('input-file-pattern, button-load', pattern, n)
    # if cache.has(pattern): return cache.set(cache.get(pattern))
    dfBpAll, dfImuAll = load_data.load_dataframe_from_mat('data/raw_mat/', pattern)
    dfAll = preprocess.merge_imu_vcg_with_heartbeats(dfBpAll, dfImuAll)
    cache.set(dfAll, key=pattern)
    return cache.set(dfAll)


### Viz Functions
@app.callback(
    Output('viz','children'),
    Input('raw-data','data'), Input('file_ids','value')
)
def trigger_viz(data, file_ids):
    log('trigger_viz', data, file_ids)

    dfAll = cache.get(data)
    # dfAll = dfAll[dfAll.file.isin(file_ids)]
    dfImu = dfAll[constants.INDICIES + constants.IMU_COLS]
    dfBp = dfAll[constants.INDICIES + constants.BP_COLS + ['patient','test_type']]

    df = summarize_data_bp(dfBp).reset_index()
    return html.Div([
        html.P(children=["BP Shape", str(dfBpAll.shape), "IMU Shape", str(dfImuAll.shape),]),
        dash_table.DataTable(
            df.to_dict('records'),
            [{"name": i, "id": i} for i in df.columns]
        ),
        # px.line(dfBpAll, y='sbp', color='file')
    ])
    
    # return dcc.Graph(
    #     figure=px.scatter_3d(dff2, 'sepal_length','petal_length','sepal_width', color='species')
    # )



### Modeling & Save
@app.callback(
    Output('model-binary','data'),
    Input('button-train','n_clicks'), State('raw-data','data')
)
def trigger_train_model(n, data):
    log('trigger_train_model',n, data)
    if (n < 1) or data == None : 
        log('trigger_train_model skipping..')
        return None
    dff = cache.get(data)
    
    model = lambda x : np.rand.rand()
    return cache.set(model)


### Run App
if __name__ == '__main__':
    app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True,
        #**{'width': '1200px', 'height': '800px'}
    )

In [ ]:
dfAll = cache.CACHE['LLV']

dfImu = dfAll[constants.INDICIES + constants.IMU_COLS]
dfBp = dfAll[constants.INDICIES + constants.BP_COLS + ['patient','test_type']]